In [ ]:
!pip install -U sentence-transformers
!pip install transformers
!pip install faiss-gpu
!pip install tqdm
!pip install swifter


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=ccc017d843d8029bcf1010f28cf0ba383f287283f566a6e340dab44774239cb2
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.6/490.6 kB 10.7 MB/s eta

In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import pipeline

import faiss
import pandas as pd
import numpy as np
from pathlib import Path
import numpy as np
from tqdm import tqdm
import swifter

tqdm.pandas()

In [ ]:
def chunkify(words: list[str], size: int):
    for i in range(0, len(words), size):
        yield " ".join(words[i : i + size])


def normalize(vector: np.ndarray) -> np.ndarray:
    norm = np.linalg.norm(vector)
    return vector / norm


def generate_chunks(
    raw_text: str,
    window: int,
    stride: int,
) -> list[str]:
    if not isinstance(raw_text, str):
      print(raw_text)
    words = raw_text.split()
    chunks = list(chunkify(words, window - stride))

    if stride > 0:
        for idx, _ in enumerate(chunks):
            if idx != len(chunks) - 1:
                chunks[idx] += chunks[idx + 1][:stride]

    return chunks

def convert(item):
    item = item.strip()  # remove spaces at the end
    item = item[1:-1]    # remove `[ ]`
    item = np.fromstring(item, sep=' ')  # convert string to `numpy.array`
    return item

Data Exploration

In [ ]:
WINDOWS_SIZE=500
STRIDE=0
ENCODER_MODEL = 'sentence-transformers/msmarco-distilbert-base-v4'
encoder_model = SentenceTransformer(ENCODER_MODEL, device="cuda")


In [ ]:
# Let Us kee only Content
import os
dir = Path(os.getcwd()) /"drive"/"MyDrive"/"Pensieve"
articles = dir / "articles.csv"
passages_file = dir / "passages.pkl"

if not passages_file.exists():
  articles = pd.read_csv(articles)
  articles = articles[articles['Content'].notna()]
  passages = articles.head(50000).copy()
  passages["Content"]= articles["Content"].apply(lambda x : generate_chunks(raw_text=x,window=WINDOWS_SIZE, stride=STRIDE))
  passages = passages.explode("Content")
  passages["p_id"] = np.arange(alen(passages))
  passages["embeddings"] = passages["Content"].progress_apply(lambda passage: encoder_model.encode(passage))
  passages["normalized_embeddings"] = passages["embeddings"].swifter.apply(lambda emb: normalize(emb))
  passages.to_pickle(passages_file)
  del articles
else:
  passages = pd.read_pickle(passages_file)

passages.count()

Unnamed: 0.1             93298
Unnamed: 0               93298
Content                  93298
id                       93298
p_id                     93298
embeddings               93298
normalized_embeddings    93298
dtype: int64

### Generate Some Questions and get Answers

In [ ]:
samples_file = dir / "samples.pkl"
if not samples_file.exists():
  samples = passages.sample(100,random_state=24).reset_index()
  pipe = pipeline("text2text-generation", "lmqg/flan-t5-base-squad-qg")
  samples["question"] = samples["Content"].swifter.apply(lambda x: pipe(f"generate query: {x[:450]}")[0].get("generated_text"))
  samples.to_pickle(samples_file)
else:
  samples = pd.read_pkl(samples_file)

Pandas Apply:   0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
samples = samples[["p_id", "normalized_embeddings", "question"]]
samples["normalized_embeddings"][0].shape


(768,)

### Build FAISS Index

In [ ]:
emb = passages["normalized_embeddings"][0]
emb.dtype

dtype('float32')

In [ ]:
from dataclasses import dataclass

@dataclass
class VectorDB:
    dims: int
    index = None

    def create_index(self):
        self.index = faiss.index_factory(self.dims, "Flat", faiss.METRIC_INNER_PRODUCT)

    def insert(self, embedding):
        if self.index is None:
            self.create_index()
        self.index.add(embedding)

    def search(self, query, k):
        if self.index is None:
            raise ValueError("Index is Empty")
        score, ref =  self.index.search(query, k)
        return list(score[0]), list(ref[0])




In [ ]:
db = VectorDB(dims=emb.shape[0])


for embeddings in passages["normalized_embeddings"]:
    emb = np.array([embeddings])
    db.insert(emb)

In [ ]:
samples["q_encode"] = samples["question"].progress_apply(lambda x: np.array([encoder_model.encode(x)]))
samples["search_result"] = samples["q_encode"].progress_apply(lambda x: db.search(x,20)[1])
samples["matched"] = samples.progress_apply(lambda x: x["p_id"] in x["search_result"], axis=1)
samples["first_matched"] = samples.progress_apply(lambda x: x["p_id"] == x["search_result"][0], axis=1)
samples

Pandas Apply:   0%|          | 0/100 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/100 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/100 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/100 [00:00<?, ?it/s]

,p_id,normalized_embeddings,question,q_encode,search_result,matched,first_matched
0,720,"[-0.01683151, -0.05189047, 0.02109172, 0.01804...",What was the name of the woman who was alleged...,"[[0.22725497, -0.45656106, -0.46068743, -0.126...","[60947, 52729, 47060, 91681, 9893, 22531, 4357...",False,False
1,69655,"[-0.026371745, -0.012039373, -0.006307815, -0....",What did Kushner do to try to jumpstart moribu...,"[[0.25268212, 0.075271145, -1.2572666, -0.2233...","[81144, 17324, 46391, 17395, 28030, 17450, 867...",True,False
2,52231,"[-0.039642997, -0.03980817, 0.0023110125, -0.0...",What did Taylor say he was asked to perform?,"[[-0.09515276, -0.066348456, -0.7718528, -0.22...","[28507, 81738, 86920, 22880, 57082, 53069, 896...",False,False
3,80300,"[0.031158775, -0.016868923, -0.07492862, -0.02...","– The Grey is more startling than frightening,...","[[-0.54827243, -0.25325033, -0.49713296, 0.142...","[21490, 80300, 80298, 79929, 27995, 42013, 279...",True,False
4,60398,"[-0.0628281, 0.0068175676, -0.0032144089, 0.02...",What is the North's fifth nuclear test?,"[[-0.43037316, -0.5688449, 0.20396103, 0.47459...","[7208, 65509, 75158, 65510, 71332, 60271, 6039...",True,False
...,...,...,...,...,...,...,...
95,84969,"[-0.008892025, -0.04063798, 0.050062984, -0.01...",What country is southern New Zealand?,"[[0.2317342, -0.9713147, 0.38536468, -0.311693...","[84969, 35133, 1064, 56700, 39097, 75038, 1068...",True,True
96,91920,"[0.0035913112, -0.036466613, 0.025070554, -0.0...",What was the name of the person who was struck...,"[[-0.16366948, -0.20204177, -1.1205294, 0.0146...","[11833, 3137, 52729, 10578, 71943, 17664, 7588...",False,False
97,88161,"[-0.018631939, -0.026312923, 0.06074842, -0.00...",What was the result of the goal of Mario Gotze...,"[[0.09955256, 0.13392389, 0.68561804, 0.372808...","[89853, 76795, 88161, 57450, 75618, 77260, 138...",True,False
98,7012,"[0.029474813, -0.015013874, -0.09521261, -0.01...",What was the first Israeli military presence o...,"[[0.2995194, -0.12091103, -1.552469, -0.124524...","[85089, 7012, 40374, 60161, 10370, 24922, 7591...",True,False


In [ ]:
samples[samples["matched"]==False].count()

p_id                     38
normalized_embeddings    38
question                 38
q_encode                 38
search_result            38
matched                  38
first_matched            38
dtype: int64